## Preprocessing script 

There are two main outputs of this script:
1) Data for leaving trial (only leave trials) 
2) Data for all trials 


In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from groo.groo import get_root
root_dir = get_root(".ed-foraging-root")

files=os.listdir(os.path.join(root_dir, 'anon-data', 'task_data', 'raw_data') )

data = pd.DataFrame()
fulldata = pd.DataFrame()
for f_idx, f in enumerate(files):
    df = pd.read_csv(os.path.join(root_dir, 'anon-data', 'task_data', 'raw_data', f))

    # Filter file for only the relevant lines 
    df = df.loc[df["decision_type"].isin([1,2])]
    df["id"] = df["PROLIFIC_PID"]
    
    df["decay_lvl"] = df["decay_rate_mean"].replace({0.81: "low", 0.84:"low", 0.91: "high", 0.96: "high"})
    df["decay_fac"] = df["decay_rate_mean"].replace({0.81: "fast", 0.84:"fast", 0.91: "slow", 0.96: "slow"})
    df["travel_time_lvl"] = df["travel_duration"].replace({2:"short", 2.5: "short", 3: "short", 9: "long"})

    data = pd.concat([data, df])


# Write full data set 
#fulldata.to_csv(os.path.join(root_dir, 'anon-data', 'task_data', 'foraging_raw_behavioural_data.csv'))

# Merge questionnaire and behavioural data 
qdata = pd.read_csv(os.path.join(root_dir, 'anon-data', 'questionnaires', 'anonymized_final_questonnaire_data.csv'))
final_ids = qdata["PROLIFICID"]
data = data.loc[data["PROLIFIC_PID"].isin(final_ids),].set_index("PROLIFIC_PID")
data = data.join(qdata.set_index("PROLIFICID"), rsuffix="_qq")
data["group"] = data["recruitment_group"]

export = 1
keep_columns = ["patch_actions_taken", "patch_time_elapsed_seconds", "blk_number", "interviewtime", "blk_time_remaining_event_onset", "patch_reward_prospective", "exploit_phase_RT", "id", "age", "gender", "diagnosis", "which_diagnosis", "EducationYears", "decay_lvl", "decay_fac", "travel_time_lvl", "decay_rate_mean", "travel_duration",  "group", "trials.thisN"]
## Filter for only leaving 
tdf = data.loc[data["exploit_phase_key"].isin(["s"]),keep_columns]
tdf=tdf.rename(columns={"exploit_phase_RT": "RT", 
                        "trials.thisN": "episode",
                        "interviewtime": "survey_duration", 
                        "block":"blk_number",
                        "patch_time_elapsed_seconds": "patch_time", 
                        "patch_reward_prospective": "reward_remaining", 
                        "blk_time_remaining_event_onset": "remaining_time"})
tdf["logRT"] = np.log(tdf["RT"])
if export == 1:
    tdf.to_csv(os.path.join(root_dir, 'anon-data', 'task_data', 'leaving_time_data.csv'))


keep_columns = ["patch_actions_taken","patch_reward_prospective", "id", "age", "gender", "EducationYears", "decay_lvl","decay_fac", "travel_time_lvl", "decay_rate_mean", "travel_duration", "blk_time_remaining_event_onset",  "group", "patch_reward_recieved", "patch_time_elapsed_seconds", "exploit_phase_RT", "decision_type", "blk_number", "trials.thisN" ]
col_mapping ={"trials.thisN": "episode", "exploit_phase_RT": "RT", "patch_time_elapsed_seconds": "patch_time", 
                        "patch_reward_prospective": "reward_remaining", 
                        "blk_time_remaining_event_onset": "remaining_time"}


tdf = data.loc[data["exploit_phase_key"].isin(["s", "space"]),keep_columns].rename(columns=col_mapping)

if export == 1:
    tdf.to_csv(os.path.join(root_dir, 'anon-data', 'task_data', 'full_beh_data.csv'))

/home/ondrej/anaconda3/envs/foraging_env/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
